<a href="https://colab.research.google.com/github/incognitorxa/EvolvEDStudyApplication/blob/main/IR_H_M_2025_Exercise2_TEMPLATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Exercise 2 Notebook


This is the template notebook for Exercise 2. The specification for the exercise and the corresponding Exercise 2 Quiz submission instance are available on the Moodle page of the course.

This exercise builds upon Exercise 1, and assumes that you are now familiar with concepts we have introduced in both the Lab 1 and Exercise 1, including:
 - [PyTerrier operators](https://pyterrier.readthedocs.io/en/latest/operators.html)
 - [Pyterrier apply transformers](https://pyterrier.readthedocs.io/en/latest/transformer.html)
 - [PyTerrier pt.Experiment()](https://pyterrier.readthedocs.io/en/latest/experiments.html)


## PyTerrier Setup

First, let's install PyTerrier as usual. We require a specific version of LightGBM. Do not change this version - if you are running locally on Apple Silicon, this wont work, and you should move back to Google Colab.

In [1]:
%pip install -q python-terrier lightgbm==2.2.3 pyterrier-caching

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.2 MB/s eta 0:00:00


Let's start PyTerrier

In [2]:
import pyterrier as pt

# we require a specific version of LightGBM for this exercise
import lightgbm
assert lightgbm.__version__ == '2.2.3'

We're going to speed things up for you by caching the PL2 results and the standard feature set.

DO NOT be tempted to cache your own feature implementations.

In [3]:
from pyterrier_caching import RetrieverCache, SparseScorerCache

CACHE=True

## Index, Topics & Qrels for Exercise 2

You will need your login & password credentials from Exercise 1. We will be using again the "50pct" and the "trec-wt-2004" datasets from Exercise 1.


In [4]:
UNAME="3046694c"
PWORD="7f154aaa"

# we will again be using the "50pct" and "trec-wt-2004" datasets
Fiftypct = pt.get_dataset("50pct",  user=UNAME, password=PWORD)
dotgov_topicsqrels = pt.get_dataset("trec-wt-2004")

On the other hand, you will be using a slightly updated index for Exercise 2. It is a bit bigger than the Exercise 1 index, hence it takes about 2-3 minutes to download to Colab.


In [5]:
indexref = Fiftypct.get_index(variant="ex3")
index = pt.IndexFactory.of(indexref, memory=True)


data.meta-0.fsomapfile:   0%|          | 0.00/50.1M [00:00<?, ?iB/s]

data-pagerank.oos:   0%|          | 0.00/9.52M [00:00<?, ?iB/s]

data.direct.bf: 0.00iB [00:00, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/19.3M [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/384M [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/6.16M [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/861M [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/3.74k [00:00<?, ?iB/s]

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started (triggered by IndexFactory.of) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


Let's check out the new index. Compared to the index we used for Exercise 1, you can see that this index has `Field Names: [TITLE, ELSE]`, which means that we can provide statistics about how many times each term occurs in the title of each document (the "TITLE" field), vs the rest of the document (the "ELSE" field). Refer to Lecture 7 for more information about fields.

Let's also display the keys in the meta index - this is the metadata that we have stored for each document. You can see that we are storing the "url" and the "body" (content) of the document. These will particularly come in handy for Q2 and Q3 of Exercise 2, respectively.


In [6]:
print(index.getCollectionStatistics())
print("In the meta index: " + str(index.getMetaIndex().getKeys()))

Number of documents: 807775
Number of terms: 2043788
Number of postings: 177737957
Number of fields: 2
Number of tokens: 572916194
Field names: [TITLE, ELSE]
Positions:   false

In the meta index: ['docno', 'url', 'title', 'body']


Finally, these are all of the topics and qrels (including the training and validation datasets) that you will need to conduct Exercise 2.

In [7]:
tr_topics = Fiftypct.get_topics("training")
va_topics = Fiftypct.get_topics("validation")

tr_qrels = Fiftypct.get_qrels("training")
va_qrels = Fiftypct.get_qrels("validation")

test_topics = dotgov_topicsqrels.get_topics("hp")
test_qrels = dotgov_topicsqrels.get_qrels("hp")

training.topics:   0%|          | 0.00/7.75k [00:00<?, ?iB/s]

validation.topics:   0%|          | 0.00/4.39k [00:00<?, ?iB/s]

training.qrels:   0%|          | 0.00/2.88k [00:00<?, ?iB/s]

validation.qrels:   0%|          | 0.00/1.48k [00:00<?, ?iB/s]

/usr/local/lib/python3.11/dist-packages/pyterrier/datasets.py:167: UserWarning: Problem fetching https://trec.nist.gov/data/web/Web2004.query.stream.trecformat.txt, resorting to next mirror
  warn(


Web2004.query.stream.trecformat.txt:   0%|          | 0.00/15.3k [00:00<?, ?iB/s]

/usr/local/lib/python3.11/dist-packages/pyterrier/datasets.py:167: UserWarning: Problem fetching https://trec.nist.gov/data/web/04.topic-map.official.txt, resorting to next mirror
  warn(


04.topic-map.official.txt:   0%|          | 0.00/2.53k [00:00<?, ?iB/s]

/usr/local/lib/python3.11/dist-packages/pyterrier/datasets.py:167: UserWarning: Problem fetching https://trec.nist.gov/data/web/04.qrels.web.mixed.txt, resorting to next mirror
  warn(


04.qrels.web.mixed.txt:   0%|          | 0.00/1.90M [00:00<?, ?iB/s]

## Baseline Setup

We introduce here the terrier.Retriever for our baseline. Note that:
 - We are using PL2 as our weighting model to generate the candidate set of documents to re-rank.
 - We expose more document metadata, namely "url" and "body" for each document retrieved, which you will need to deploy your two new features.
 - By setting `verbose=True`, we display a progress bar while retrieval executes.
 - We cache PL2 to make it faster for reuse in later experiments.

In [8]:
firstpass = pt.terrier.Retriever(index, wmodel="PL2", metadata=["docno", "url", "body"], verbose=True)
if CACHE: # wrap in a cache transformer
    firstpass = RetrieverCache('pl2-cache', firstpass)

Let's see the resulting output - you can see that there are now "url" and "body" attributes for each retrieved document. (We also display a progress bar, enabled by the `verbose=True`).

In [9]:
firstpass.search("chemical reactions")

TerrierRetr(PL2): 0q [00:00, ?q/s]

TerrierRetr(PL2):   0%|          | 0/1 [00:00<?, ?q/s]

BitPostingIndex loading /root/.pyterrier/corpora/50pct/index/ex3/data.inverted.bf to memory


,qid,docid,docno,url,body,rank,score,query
0,1,513586,G18-38-1767991,http://www.boulder.nist.gov/div838/tar/file03....,NIST - Physical and Chemical Properties Divi...,0,12.755546,chemical reactions
1,1,38544,G01-14-2537005,http://www.labtrain.noaa.gov/shemtfa/chemhaz/n...,. \r\n \r\n \r\n \...,1,11.906524,chemical reactions
2,1,707122,G26-06-3754605,http://www.aps.anl.gov/xfd/tech/safetyenvelope...,APS Experiment Safety Envelope 6: Chemicals \r...,2,11.877550,chemical reactions
3,1,382754,G13-59-3981168,http://response.restoration.noaa.gov/chemaids/...,"""); }\n else {\n document.wr...",3,11.858475,chemical reactions
4,1,70292,G02-16-2617043,http://www.symp14.nist.gov/PDF/COR04MAY.PDF,A Database of Chemical Reactions Designed to A...,4,11.731490,chemical reactions
...,...,...,...,...,...,...,...,...
995,1,246965,G08-68-4141101,http://en-env.llnl.gov/asd/pinatub.html,The Chemical and Radiative Effects of the Moun...,995,6.290707,chemical reactions
996,1,611136,G22-04-3955177,http://eospso.gsfc.nasa.gov/ftp_docs/Ch7.pdf,Chapter 7\n ...,996,6.289822,chemical reactions
997,1,594957,G21-38-0191596,http://www.oit.doe.gov/news/oittimes/wn02/wn02...,search \r\n\t\t\t\t\t\t\t\t \r\n\t\t\t\t\t\t\...,997,6.287830,chemical reactions
998,1,280944,G09-85-3411646,http://www.ig.doe.gov/pdf/chemfina.pdf,INS-O-00-01\n\n\n\n\n\n\n\n\n I N S P E C T...,998,6.287759,chemical reactions


# Standard List of Features

Let's introduce the list of features we need to deploy a baseline learning-to-rank approach.

We again cache the results of FeaturesRetriever to make it faster.

In [10]:
pagerankfile = indexref + "/data-pagerank.oos"

# DO *NOT* CHANGE THIS LIST. Use PyTerrier operators to add features...
features = [
    "SAMPLE", #ie PL2 - this exposes the scores used to obtain the candidate set as a feature
    "WMODEL:SingleFieldModel(BM25,0)", #BM25 title
    "QI:StaticFeature(OIS,%s)" % pagerankfile,
]

stdfeatures = pt.terrier.FeaturesRetriever(index, features, verbose=True)
if CACHE: # wrap in a cache transformer
    stdfeatures = SparseScorerCache('features-cache', stdfeatures, value="features", pickle=True, verbose=True)

stage12 = firstpass >> stdfeatures

This is our feature set. We will be using FeaturesBatchRetrieve to compute these extra features on the fly. Let's see the output. You can see that there is now a "features" column.

In [11]:
stage12.search("chemical reactions").head(2)

TerrierFeatRetr(3 features):   0%|          | 0/1 [00:00<?, ?q/s]

Sqlite3ScorerCache('features-cache', TerrierFeatRetr(/root/.pyterrier/corpora/50pct/index/ex3/data.properties,['SAMPLE', 'WMODEL:SingleFieldModel(BM25,0)', 'QI:StaticFeature(OIS,/root/.pyterrier/corpora/50pct/index/ex3/data-pagerank.oos)'],{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'matching': 'FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull', 'decorate_batch': 'on'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.c

,qid,docid,docno,url,body,rank,score,query,features
0,1,513586,G18-38-1767991,http://www.boulder.nist.gov/div838/tar/file03....,NIST - Physical and Chemical Properties Divi...,0,12.755546,chemical reactions,"[12.755545561073266, 3.0924078763629836, 0.000..."
1,1,38544,G01-14-2537005,http://www.labtrain.noaa.gov/shemtfa/chemhaz/n...,. \r\n \r\n \r\n \...,1,11.906524,chemical reactions,"[11.90652405775751, 10.789390732195702, 0.0002..."


Let's look in more detail at the features. It is clear that there are 3 numbers for each document. The first is the PL2 score (1.27555456e+01 == 12.7555), the second is the BM25 score, and the third is the PageRank (a link analysis feature - discussed in more detail in Lecture 9)

In [12]:
stage12.search("chemical reactions").head(1).iloc[0]["features"]

Sqlite3ScorerCache('features-cache', TerrierFeatRetr(/root/.pyterrier/corpora/50pct/index/ex3/data.properties,['SAMPLE', 'WMODEL:SingleFieldModel(BM25,0)', 'QI:StaticFeature(OIS,/root/.pyterrier/corpora/50pct/index/ex3/data-pagerank.oos)'],{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'matching': 'FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull', 'decorate_batch': 'on'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.c

array([1.27555456e+01, 3.09240788e+00, 1.05668333e-04])

# Q1

You now have everyting you need to attempt Q1. You will need to refer to the specification, and to PyTerrier's [learning to rank documentation](https://pyterrier.readthedocs.io/en/latest/ltr.html).

You should use a LightGBM LambdaMART implementation (*not* XGBoost), instantiated using the configuration suggested in the PyTerrier documentation.

Hints:
 - You will need to use the provided separate “training” and “validation” topic sets and qrels to train the learning-to-rank.
 - There is no need to vary the configuration of LightGBM from that in the documentation.
 - Training and evaluating a LTR pipeline takes around 5 minutes.

In [13]:
train_pipeline = firstpass >> stdfeatures
valid_pipeline = firstpass >> stdfeatures
train_data = train_pipeline.transform(tr_topics)
valid_data = valid_pipeline.transform(va_topics)
# Merge qrels (relevance labels) safely
train_data = train_data.merge(tr_qrels, on=['qid', 'docno'], how='left').fillna(0)
valid_data = valid_data.merge(va_qrels, on=['qid', 'docno'], how='left').fillna(0)
# Ensure "label" is properly formatted
train_data["label"] = train_data["label"].astype(int)
valid_data["label"] = valid_data["label"].astype(int)

# Debug: Check column names after merging
print("Train Data Columns AFTER merge:", train_data.columns)
print("Validation Data Columns AFTER merge:", valid_data.columns)

import lightgbm as lgb
import pickle

# Define LightGBM model
lmart = lgb.LGBMRanker(
    task="train",
    objective="lambdarank",
    metric="ndcg",
    num_leaves=100,
    learning_rate=0.05,
    n_estimators=100,
    verbosity=-1
)

# Ensure features are in correct format
import numpy as np

# Convert features to 2D NumPy arrays
train_X = np.vstack(train_data["features"].to_numpy())
train_y = train_data["label"].astype(int)
train_q = train_data.groupby("qid").size().to_numpy()

valid_X = np.vstack(valid_data["features"].to_numpy())
valid_y = valid_data["label"].astype(int)
valid_q = valid_data.groupby("qid").size().to_numpy()

train_data["features"] = train_data["features"].apply(lambda x: np.array(x, dtype=np.float32))
valid_data["features"] = valid_data["features"].apply(lambda x: np.array(x, dtype=np.float32))


# Train the model
lmart.fit(
    train_X, train_y, group=train_q,
    eval_set=[(valid_X, valid_y)],
    eval_group=[valid_q],
    eval_metric="ndcg",
    verbose=10
)

# Save the trained model
with open("ltr_model.pkl", "wb") as f:
    pickle.dump(lmart, f)

print("Model trained successfully!")
###################################################################
# Run experiment with perquery=True to enable t-test
from scipy.stats import ttest_rel
results_q1 = pt.Experiment(
    [ltr_pipeline_3f, firstpass],
    test_topics,
    test_qrels,
    eval_metrics=["map", "P_5"],
    names=["LTR", "PL2"],
    perquery=True,
    dataframe=True  # Required to get long format
)

print("\nResults Columns:", results_q1.columns)
print("\nSample Row:\n", results_q1.iloc[0])

###################################################################
# Separate per-metric results
map_results = results_q1[results_q1["measure"] == "map"]
p5_results = results_q1[results_q1["measure"] == "P_5"]

# Pivot to wide format: one row per topic (qid), columns per system
map_wide = map_results.pivot(index="qid", columns="name", values="value")
p5_wide = p5_results.pivot(index="qid", columns="name", values="value")

# Compute mean metrics
mean_map_ltr = map_wide["LTR"].mean()
mean_map_pl2 = map_wide["PL2"].mean()

mean_p5_ltr = p5_wide["LTR"].mean()
mean_p5_pl2 = p5_wide["PL2"].mean()

print("\nMean Metrics Summary:")
print(f"MAP  - LTR: {mean_map_ltr:.4f}, PL2: {mean_map_pl2:.4f}")
print(f"P@5  - LTR: {mean_p5_ltr:.4f}, PL2: {mean_p5_pl2:.4f}")

###################################################################
# Paired t-tests
pval_map = ttest_rel(map_wide["LTR"], map_wide["PL2"]).pvalue
pval_p5 = ttest_rel(p5_wide["LTR"], p5_wide["PL2"]).pvalue

print(f"\nT-test p-value (MAP): {pval_map:.6f}")
print(f"T-test p-value (P@5): {pval_p5:.6f}")

TerrierRetr(PL2): 0q [00:00, ?q/s]

TerrierRetr(PL2):   0%|          | 0/96 [00:00<?, ?q/s]

TerrierFeatRetr(3 features):   0%|          | 0/96 [00:00<?, ?q/s]

Sqlite3ScorerCache('features-cache', TerrierFeatRetr(/root/.pyterrier/corpora/50pct/index/ex3/data.properties,['SAMPLE', 'WMODEL:SingleFieldModel(BM25,0)', 'QI:StaticFeature(OIS,/root/.pyterrier/corpora/50pct/index/ex3/data-pagerank.oos)'],{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'matching': 'FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull', 'decorate_batch': 'on'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.c

TerrierRetr(PL2): 0q [00:00, ?q/s]

...

TerrierFeatRetr(3 features):   0%|          | 0/54 [00:00<?, ?q/s]

Sqlite3ScorerCache('features-cache', TerrierFeatRetr(/root/.pyterrier/corpora/50pct/index/ex3/data.properties,['SAMPLE', 'WMODEL:SingleFieldModel(BM25,0)', 'QI:StaticFeature(OIS,/root/.pyterrier/corpora/50pct/index/ex3/data-pagerank.oos)'],{'terrierql': 'on', 'parsecontrols': 'on', 'parseql': 'on', 'applypipeline': 'on', 'localmatching': 'on', 'filters': 'on', 'decorate': 'on', 'matching': 'FatFeaturedScoringMatching,org.terrier.matching.daat.FatFull', 'decorate_batch': 'on'},{'querying.processes': 'terrierql:TerrierQLParser,parsecontrols:TerrierQLToControls,parseql:TerrierQLToMatchingQueryTerms,matchopql:MatchingOpQLParser,applypipeline:ApplyTermPipeline,context_wmodel:org.terrier.python.WmodelFromContextProcess,localmatching:LocalManager$ApplyLocalMatching,qe:QueryExpansion,labels:org.terrier.learning.LabelDecorator,filters:LocalManager$PostFilterProcess,decorate:SimpleDecorateProcess', 'querying.postfilters': 'decorate:SimpleDecorate,site:SiteFilter,scope:Scope', 'querying.default.c

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[10]	valid_0's ndcg@1: 0.388889
[20]	valid_0's ndcg@1: 0.407407
[30]	valid_0's ndcg@1: 0.351852
[40]	valid_0's ndcg@1: 0.333333
[50]	valid_0's ndcg@1: 0.351852
[60]	valid_0's ndcg@1: 0.37037
[70]	valid_0's ndcg@1: 0.37037
[80]	valid_0's ndcg@1: 0.37037
[90]	valid_0's ndcg@1: 0.333333
[100]	valid_0's ndcg@1: 0.333333
Model trained successfully!


NameError: name 'ltr_pipeline_3f' is not defined

# Q2 - URL Length Features

In this block, please provide your code for Q2 concerning your two URL Length features, namely URL Length by counting slashes (URL-slashes) and URL Length through using the type of the URL (URL-type). The two different URL length features that you will need to implement are detailed in the specification. Do carefully read and follow the Exercise 2 specification before starting the implementation of the features.

Some hints:

 - For computing each of your URL features, you will need to use an appropriate [pt.apply function](https://pyterrier.readthedocs.io/en/latest/apply.html). The dataframe of results obtained from the `firstpass` transformer has all of the information you need. You can see how fast your apply function is by setting `verbose=True`.

 - You can use the `**` PyTerrier operator for combining feature sets.

 - Refer to the PyTerrier learning to rank documentation  concerning `feature_importances_` for obtaining feature importances.

 - You may wish to refer to Python's [`urlparse()`](https://docs.python.org/3/library/urllib.parse.html) function.

 - Use Python assertions to test that your feature implmentation(s) give the expected results. **Remember that you need to report along your code all the tests you have conducted to ascertain the code's correctness.**


## Q2 (a) URL-Slashes Feature

In this block you should define your URL-Slashes feature, and **test it**. **Show clearly all the tests** that you have conducted to test that your feature works as expected.

In [ ]:
import pandas as pd
from urllib.parse import urlparse

# --- URL-Slashes Feature Function ---
def url_slashes_feature(df):
    """
    Adds a feature counting the number of slash ('/') characters in the URL path.

    Parameters:
    df (pd.DataFrame): Input dataframe with a 'url' column.

    Returns:
    pd.DataFrame: Output dataframe with columns ['qid', 'docno', 'url_slashes'].
    """
    def count_slashes(url):
        if pd.isna(url):
            return 0
        parsed = urlparse(url)
        path = parsed.path
        return path.count('/')

    df = df.copy()
    df['url_slashes'] = df['url'].apply(count_slashes)
    return df[['qid', 'docno', 'url_slashes']]

# Wrap as PyTerrier Transformer
url_slashes_transformer = pt.apply.by_query(url_slashes_feature, verbose=True)

# --- Unit Tests ---
# Create sample test data
test_df = pd.DataFrame({
    'qid': [1, 1, 1],
    'docno': ['D1', 'D2', 'D3'],
    'url': [
        'http://example.com/',
        'https://example.com/about/team',
        'http://example.com/section/page/item/'
    ]
})

# Apply the feature function
test_result = url_slashes_feature(test_df)

# Validate correctness using assertions
assert test_result.loc[0, 'url_slashes'] == 1   # '/' at the end
assert test_result.loc[1, 'url_slashes'] == 2   # '/about/team'
assert test_result.loc[2, 'url_slashes'] == 4   # '/section/page/item/'

print("URL-Slashes feature passed all tests.")
print(test_result)

#### (i) URL-Slashes as a PL2 re-ranker

Now you should evaluate your URL-slashes score by re-ranking PL2, without applying learning-to-rank.

Hint:
 - Your reranker should order documents in descending order, i.e. longest URLs first.

 You can now answer the corresponding quiz questions.

In [ ]:
from urllib.parse import urlparse

def add_url_slashes(df):
    df = df.copy()
    df["url_slashes"] = df["url"].apply(lambda u: urlparse(u).path.count('/'))
    return df

url_slashes_transformer = pt.apply.generic(add_url_slashes, verbose=True)

# Reranker (sort descending by slashes)
url_slashes_reranker = pt.apply.by_query(
    lambda df: df.sort_values("url_slashes", ascending=False),
    add_ranks=False,  # <-- make sure this is False
    verbose=True
)

# Full pipeline
pipeline_slash = firstpass >> url_slashes_transformer >> url_slashes_reranker
cryption_query = pd.DataFrame([["q_cryption", "cryption"]], columns=["qid", "query"])
cryption_results = pipeline_slash.transform(cryption_query)

# Get the top-ranked document
top_docno = cryption_results.iloc[0]['docno']
print("Top-ranked docno for 'cryption':", top_docno)


# Run experiment
pt.Experiment(
    [pipeline_slash],
    test_topics,
    test_qrels,
    eval_metrics=["map", "ndcg", "P.10"],
    names=["URL-Slashes"]
)

#### (ii) URL-Slashes within an LTR model

Now you should evaluate your URL-slashes score as a feature within a new learned model.

Hint:
 - Carefully consider how to integrate your feature into an LTR model, based on your understanding of how a regression tree works.

You can now answer the corresponding quiz questions.

In [ ]:

train_slashes = url_slashes_transformer.transform(train_data)
valid_slashes = url_slashes_transformer.transform(valid_data)

# Combine with existing features
train_data["features"] = train_data["features"].apply(np.array)
valid_data["features"] = valid_data["features"].apply(np.array)

# Append the new feature to the existing features (PL2, BM25, PageRank + URL-Slashes)
train_data["features"] = [
    np.append(old, [new]) for old, new in zip(train_data["features"], train_slashes["url_slashes"])
]
valid_data["features"] = [
    np.append(old, [new]) for old, new in zip(valid_data["features"], valid_slashes["url_slashes"])
]

lmart_slash = lgb.LGBMRanker(
    task="train",
    objective="lambdarank",
    metric="ndcg",
    num_leaves=100,
    learning_rate=0.05,
    n_estimators=100,
    verbosity=-1
)

train_X = np.vstack(train_data["features"].to_numpy())
train_y = train_data["label"].astype(int)
train_q = train_data.groupby("qid").size().to_numpy()

valid_X = np.vstack(valid_data["features"].to_numpy())
valid_y = valid_data["label"].astype(int)
valid_q = valid_data.groupby("qid").size().to_numpy()

lmart_slash.fit(
    train_X, train_y, group=train_q,
    eval_set=[(valid_X, valid_y)],
    eval_group=[valid_q],
    eval_metric="ndcg",
    verbose=10
)

print("LTR model with URL-Slashes trained successfully!")


## Q2 (b) URL Type Feature

In this block you should define your URL Type feature and **test it**. **Show clearly all the tests** you have conducted to test that your feature works as expected.

In [ ]:
import os

# Define type mapping
EXT_TYPE_MAP = {
    'html': 1, 'htm': 1, 'php': 1, 'asp': 1, 'aspx': 1, 'jsp': 1,
    'pdf': 2,
    'jpg': 3, 'jpeg': 3, 'png': 3, 'gif': 3, 'bmp': 3, 'svg': 3,
    'mp4': 4, 'avi': 4, 'mov': 4, 'wmv': 4, 'flv': 4, 'mkv': 4,
    'csv': 5, 'xls': 5, 'xlsx': 5, 'tsv': 5, 'json': 5, 'xml': 5,
    # Add more if needed
}

def get_url_type(url):
    if pd.isna(url):
        return 0
    parsed = urlparse(url)
    path = parsed.path
    ext = os.path.splitext(path)[1].lower().replace('.', '')
    return EXT_TYPE_MAP.get(ext, 0)  # 0 if not in the map

def url_type_feature(df):
    df = df.copy()
    df['url_type'] = df['url'].apply(get_url_type)
    return df[['qid', 'docno', 'url_type']]
test_df = pd.DataFrame({
    'qid': [1]*5,
    'docno': ['D1', 'D2', 'D3', 'D4', 'D5'],
    'url': [
        'http://example.com/index.html',
        'https://example.com/doc.pdf',
        'http://example.com/image.jpg',
        'http://example.com/video.mp4',
        'http://example.com/folder/'  # no extension
    ]
})

test_result = url_type_feature(test_df)


assert test_result.loc[0, 'url_type'] == 1  # .html → 1
assert test_result.loc[1, 'url_type'] == 2  # .pdf → 2
assert test_result.loc[2, 'url_type'] == 3  # .jpg → 3
assert test_result.loc[3, 'url_type'] == 4  # .mp4 → 4
assert test_result.loc[4, 'url_type'] == 0  # no extension

print("URL-Type feature passed all tests.")
print(test_result)



#### (i) URL Type as a PL2 re-ranker

Now you should evaluate your URL type score by re-ranking PL2, without applying learning-to-rank.

Hint:
 - Your reranker should order documents in descending order, i.e. longest URLs first.

You can now answer the corresponding quiz questions.

In [ ]:
# URL-Type feature application (reusing earlier transformer if not redefined)
def get_url_type_score(url):
    path = urlparse(url).path.lower()
    if path.endswith(('.html', '.htm', '.php', '.asp')):
        return 1  # webpage
    elif path.endswith(('.pdf', '.doc', '.docx', '.ppt', '.pptx', '.xls', '.xlsx')):
        return 2  # document
    elif path.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg', '.webp')):
        return 3  # image
    elif path.endswith(('.mp4', '.avi', '.mov', '.wmv', '.flv', '.mkv')):
        return 4  # video
    else:
        return 0  # unknown or directory

def apply_url_type(df):
    df = df.copy()
    df["url_type"] = df["url"].apply(get_url_type_score)
    return df

url_type_transformer = pt.apply.generic(apply_url_type, verbose=True)

# Rerank descending by url_type
url_type_reranker = pt.apply.by_query(
    lambda df: df.sort_values("url_type", ascending=False),
    add_ranks=False,
    verbose=True
)

# Create pipeline
pipeline_url_type = firstpass >> url_type_transformer >> url_type_reranker

# Evaluate
pt.Experiment(
    [pipeline_url_type],
    test_topics,
    test_qrels,
    eval_metrics=["map", "ndcg", "P.10"],
    names=["URL-Type"]
)

#### (ii) URL Type within an LTR model

Now you should evaluate your URL type score as a feature within a new learned model.

Hint:
 - Carefully consider how to integrate your feature into an LTR model, based on your understanding of how a regression tree works.

You can now answer the corresponding quiz questions.

In [ ]:
import numpy as np
import pandas as pd
from urllib.parse import urlparse

def min_dist(query_terms, doc_terms):
    """
    Computes the minimum distance between any two occurrences of different query terms in the document.
    Returns a large finite value (1000) if no valid distances are found.
    """
    positions = {term: [] for term in query_terms}  # Store positions of query terms in the doc

    # Store term positions
    for i, term in enumerate(doc_terms):
        if term in positions:
            positions[term].append(i)

    min_distances = []

    # Compute min distances between each pair of query terms
    for i, term1 in enumerate(query_terms):
        for term2 in query_terms[i + 1:]:
            if positions[term1] and positions[term2]:  # Ensure both terms exist
                min_distance = min(abs(p1 - p2) for p1 in positions[term1] for p2 in positions[term2])
                min_distances.append(min_distance)

    # Ensure we **never return inf** and always return a numeric value
    return min(min_distances) if min_distances else 1000

def avgmindist(query: str, document: str) -> float:
    """
    Computes the average minimum distance between query term pairs in the document.
    If no valid distances are found, it returns 1000 (a large finite value).
    """
    query_terms = query.lower().split()
    doc_terms = document.lower().split()

    if len(query_terms) < 2 or len(doc_terms) == 0:  # Check for empty documents
        return 1000  # Large fallback value

    min_dists = []

    for i, term1 in enumerate(query_terms):
        for term2 in query_terms[i + 1:]:
            min_dist_value = min_dist([term1, term2], doc_terms)
            min_dist_value = min(min_dist_value, 1000)  # Ensure `inf` never happens
            min_dists.append(min_dist_value)

    return np.mean(min_dists) if min_dists else 1000  # Ensure finite output


# PyTerrier Transformer
def add_proximity_feature(df):
    df["proximity"] = df.apply(lambda row: avgmindist(row["query"], row["body"]), axis=1)
    return df[["qid", "docno", "proximity"]]

# Test Cases
test_df = pd.DataFrame({
    "qid": [1, 1, 1],
    "docno": ["D1", "D2", "D3"],
    "query": ["chemical reaction", "chemical reaction", "chemical reaction"],
    "body": [
        "Chemical reactions are crucial in science.",  # Terms are far apart
        "Reaction chemical occurs fast.",  # Terms are adjacent
        "This is a chemical-based reaction process."  # Terms have medium proximity
    ]
})

test_result = add_proximity_feature(test_df)

# Test cases
assert test_result.loc[0, "proximity"] == 1000  # Terms not both present
assert test_result.loc[1, "proximity"] == 1  # Terms adjacent
assert test_result.loc[2, "proximity"] > 1  # Medium distance

print("Proximity feature passed all tests.")
print(test_result)


# Q3 Proximity Search Feature

Now you will implement a new query-dependent feature, using the MinDist() function, as discussed in the specification. Do carefully **read the Exercise 2 specification** before starting the implementation.

Hints:
 - Again, remember to use assertions to **test** your feature implementations.
 - Refer to the PyTerrier learning to rank documentation concerning `features_importances_` for obtaining feature importances
 - For tokenisation of queries and documents, you can simply use Python's [`str.split()`](https://docs.python.org/3.3/library/stdtypes.html#str.split), without any arguments. Do not use any external libraries.

As mentioned in the specification, you should implement a function called `avgmindist()`, which takes the text of the query and the text of the document, and returns a score for the document, i.e. it must conform to the following Python specification:
```python
def avgmindist(query : str, document : str) -> float
```

**NB**: There are particular specific requirements for your implementations of MinDist() and avgmindist() that are detailed in the specification.

In [ ]:
import numpy as np

def min_dist(query_terms, doc_terms):
    """
    Computes the minimum distance between any two occurrences of different query terms in the document.
    """
    positions = {term: [] for term in query_terms}  # Store positions of query terms in the doc

    for i, term in enumerate(doc_terms):
        if term in positions:
            positions[term].append(i)

    min_distances = []

    for i, term1 in enumerate(query_terms):
        for term2 in query_terms[i + 1:]:
            if positions[term1] and positions[term2]:  # Ensure both terms exist
                min_distances.append(
                    min(abs(p1 - p2) for p1 in positions[term1] for p2 in positions[term2])
                )

    return min(min_distances) if min_distances else float('inf')

def avgmindist(query: str, document: str) -> float:
    """
    Computes the average minimum distance between query term pairs in the document.
    """
    query_terms = query.lower().split()
    doc_terms = document.lower().split()

    if len(query_terms) < 2:
        return float('inf')  # Avoid computing on single-term queries

    min_dists = []

    for i, term1 in enumerate(query_terms):
        for term2 in query_terms[i + 1:]:
            min_dists.append(min_dist([term1, term2], doc_terms))

    return np.mean(min_dists) if min_dists else float('inf')

# PyTerrier Transformer
def add_proximity_feature(df):
    df["proximity"] = df.apply(lambda row: avgmindist(row["query"], row["body"]), axis=1)
    return df[["qid", "docno", "proximity"]]

proximity_transformer = pt.apply.by_query(add_proximity_feature, add_ranks=False, verbose=True)


# Test Cases
test_df = pd.DataFrame({
    "qid": [1, 1, 1],
    "docno": ["D1", "D2", "D3"],
    "query": ["chemical reaction", "chemical reaction", "chemical reaction"],
    "body": [
        "Chemical reactions are crucial in science.",  # Terms are far apart
        "Reaction chemical occurs fast.",  # Terms are adjacent
        "This is a chemical-based reaction process."  # Terms have medium proximity
    ]
})

test_result = add_proximity_feature(test_df)

assert test_result.loc[0, "proximity"] == float('inf')  # Terms not both present
assert test_result.loc[1, "proximity"] == 1  # Terms adjacent
assert test_result.loc[2, "proximity"] > 1  # Medium distance

print("Proximity feature passed all tests.")
print(test_result)

You should test your impementation yourself (your code must list along your code *all* the test cases you deployed to test that your feature works as expected). In addition, to also allow us to verify your implementation, we have created 9 testcases. Please run `run_test_cases()` and use its responses to answer the relevant quiz questions.

Hint:
 - Our test cases took around 1-3ms each. If the testing of your implementation takes magnitudes of time longer, then this will impact upon how long it takes you to train and evaluate your implementation within a LTR pipeline.


In [ ]:
#DO NOT ALTER THIS CELL
TEST_CASES = [
  ('fermilab directory', 45, 567257), #1
  ('webcam', 45, 567257), #2
  ('DOM surface', 384034, 388292), #3
  ('DOM surface', 45, 384034), #4
  ('DOM surface document', 388292, 384034), #5
  ('DOM software AMANDA', 639302, 384034), #6
  ('fermilab directory', 388292, 384034), #7
  ('trigger data', 596532, 639302), #8
  ('underlying hardware', 384034, 333649) #9
]

def run_test_cases():
  import datetime
  docno=0
  body=3
  for i, (query, docid1, docid2) in enumerate(TEST_CASES):
    start = datetime.datetime.now()
    meta1 = index.getMetaIndex().getAllItems(docid1)
    meta2 = index.getMetaIndex().getAllItems(docid2)
    s1 = avgmindist(query, meta1[body])
    s2 = avgmindist(query, meta2[body])
    if s1 > s2:
      result = meta1[docno]
      cmpD = "%s > %s" % (meta1[docno],meta2[docno])
    elif s2 > s1:
      result = meta2[docno]
      cmpD = "%s > %s" % (meta2[docno],meta1[docno])
    else:
      result = "EQUAL"
      cmpD = "%s == %s" % (meta1[docno],meta2[docno])
    end = datetime.datetime.now()
    print("TEST CASE %d result %s time %d ms" % (i+1, result, float((end-start).microseconds)/1000.))

run_test_cases()

You should now integrate your avgmindist() function into a new LTR model, and compare its MAP & P@5 performance to the LTR baseline. You can now answer the corresponding quiz questions.

In [ ]:
def add_proximity_feature(df):
    df = df.copy()
    df["features"] = df.apply(
        lambda row: np.array([avgmindist(row["query"], row["body"])], dtype=np.float32),
        axis=1
    )
    return df[["qid", "docno", "features"]]

# Define the proximity transformer for PyTerrier
proximity_transformer = pt.apply.by_query(add_proximity_feature, add_ranks=False, verbose=True)

# Step 1: Retrieve documents using firstpass
retrieved_train = firstpass.transform(tr_topics)
retrieved_valid = firstpass.transform(va_topics)

# Step 2: Combine standard features with proximity feature using FeatureUnion
stage_features_proximity = stdfeatures ** proximity_transformer

# Step 3: Generate training and validation data with combined features
train_data_with_proximity = stage_features_proximity.transform(retrieved_train)
valid_data_with_proximity = stage_features_proximity.transform(retrieved_valid)

# Step 4: Merge with qrels to get relevance labels
train_data_with_proximity = train_data_with_proximity.merge(tr_qrels, on=['qid', 'docno'], how='left').fillna(0)
valid_data_with_proximity = valid_data_with_proximity.merge(va_qrels, on=['qid', 'docno'], how='left').fillna(0)

# Step 5: Ensure 'label' column is int
train_data_with_proximity["label"] = train_data_with_proximity["label"].astype(int)
valid_data_with_proximity["label"] = valid_data_with_proximity["label"].astype(int)

# Step 6: Extract feature matrices
train_features_with_proximity = np.vstack(train_data_with_proximity["features"].to_numpy())
valid_features_with_proximity = np.vstack(valid_data_with_proximity["features"].to_numpy())

# 🔥 Fix infinite values by replacing inf with a large finite value (e.g., 1000)
train_features_with_proximity = np.where(np.isinf(train_features_with_proximity), 1000, train_features_with_proximity)
valid_features_with_proximity = np.where(np.isinf(valid_features_with_proximity), 1000, valid_features_with_proximity)

# Step 7: Prepare labels and group info
train_labels_with_proximity = train_data_with_proximity["label"]
valid_labels_with_proximity = valid_data_with_proximity["label"]
train_groups_with_proximity = train_data_with_proximity.groupby("qid").size().to_numpy()
valid_groups_with_proximity = valid_data_with_proximity.groupby("qid").size().to_numpy()

# Step 8: Train LambdaMART model
lmart_with_proximity = lgb.LGBMRanker(
    task="train",
    objective="lambdarank",
    metric="ndcg",
    num_leaves=100,
    learning_rate=0.05,
    n_estimators=100,
    verbosity=-1
)

lmart_with_proximity.fit(
    train_features_with_proximity, train_labels_with_proximity, group=train_groups_with_proximity,
    eval_set=[(valid_features_with_proximity, valid_labels_with_proximity)],
    eval_group=[valid_groups_with_proximity],
    eval_metric="ndcg",
    verbose=10
)

print("LTR model with Proximity Feature trained successfully!")
print("All steps executed successfully.")


# Q4 A 5-feature Learning-to-Rank Model

You will now experiment with the LightGBM LambdaMART technique where you include both your added features (URL Type and AvgMinDist) along with the 3 initial features inc the initial PL2 candidate set (5 features in total).

You need to learn a *new* model when using your final selection of 5 features.

Evaluate the performance of your resulting LTR system in comparison to the LTR baseline and answer the quiz questions. For ease of comparison and readability, you should also display your results for the performance of the 4-feature LTR models.

In [ ]:
def add_proximity_feature(df):
    df = df.copy()
    proximity_values = df.apply(lambda row: avgmindist(row["query"], row["body"]), axis=1)

    if "features" in df.columns:
        df["features"] = [np.append(f, [prox]) for f, prox in zip(df["features"], proximity_values)]
    else:
        df["features"] = [np.array([prox], dtype=np.float32) for prox in proximity_values]

    return df[["qid", "docno", "features"]]

proximity_transformer = pt.apply.by_query(add_proximity_feature, add_ranks=False, verbose=True)

# Define URL type transformer with 'features' column
def get_url_type(docno):
    docno = docno.lower()
    if ".pdf" in docno:
        return 1
    elif ".jpg" in docno or ".png" in docno:
        return 2
    elif ".mp4" in docno or ".avi" in docno:
        return 3
    elif docno.endswith("/"):
        return 4
    else:
        return 0

def url_type_feature(df):
    df = df.copy()
    url_vals = df["docno"].apply(get_url_type)

    if "features" in df.columns:
        df["features"] = [np.append(f, [url]) for f, url in zip(df["features"], url_vals)]
    else:
        df["features"] = [np.array([url], dtype=np.float32) for url in url_vals]

    return df[["qid", "docno", "features"]]

urltype_transformer = pt.apply.by_query(url_type_feature, add_ranks=False, verbose=True)

# Step 1: Retrieve documents
retrieved_train = firstpass.transform(tr_topics)
retrieved_valid = firstpass.transform(va_topics)

# Step 2: Combine std + proximity + urltype (5 features total)
pipeline_5f = stdfeatures ** proximity_transformer ** urltype_transformer

train_data_5f = pipeline_5f.transform(retrieved_train)
valid_data_5f = pipeline_5f.transform(retrieved_valid)

# Step 3: Merge with qrels and prepare labels
train_data_5f = train_data_5f.merge(tr_qrels, on=['qid', 'docno'], how='left').fillna(0)
valid_data_5f = valid_data_5f.merge(va_qrels, on=['qid', 'docno'], how='left').fillna(0)

train_data_5f["label"] = train_data_5f["label"].astype(int)
valid_data_5f["label"] = valid_data_5f["label"].astype(int)

# Step 4: Combine all 5 features into training arrays
train_features_5f = np.vstack(train_data_5f["features"].to_numpy())
valid_features_5f = np.vstack(valid_data_5f["features"].to_numpy())

train_labels_5f = train_data_5f["label"]
valid_labels_5f = valid_data_5f["label"]
train_groups_5f = train_data_5f.groupby("qid").size().to_numpy()
valid_groups_5f = valid_data_5f.groupby("qid").size().to_numpy()

# Step 5: Train LambdaMART model with 5 features
lmart_5f = lgb.LGBMRanker(
    task="train",
    objective="lambdarank",
    metric="ndcg",
    num_leaves=100,
    learning_rate=0.05,
    n_estimators=100,
    verbosity=-1
)

lmart_5f.fit(
    train_features_5f, train_labels_5f, group=train_groups_5f,
    eval_set=[(valid_features_5f, valid_labels_5f)],
    eval_group=[valid_groups_5f],
    eval_metric="ndcg",
    verbose=10
)

print(" LTR model with 5 features (Proximity + URL Type) trained successfully!");
###########################Testing############################
def predict_with_lgb(df):
    df = df.copy()
    X = np.vstack(df["features"].to_numpy())

    df["score"] = lmart_5f.predict(X)
    return df  # full DataFrame so PyTerrier sees qid, docno, score
model_predict_transformer = pt.apply.generic(predict_with_lgb)

pipeline_eval_5f = (
    firstpass >>
    stdfeatures >>
    proximity_transformer >>
    urltype_transformer >>
    model_predict_transformer
)

results_5f = pt.Experiment(
    [pipeline_eval_5f],
    test_topics,
    test_qrels,
    eval_metrics=["map", "P_5"],
    names=["LTR (5 features)"]
)

print(results_5f)


# That's all Folks

**Submission Instructions:** Complete this notebook. All your answers to Exercise 2 must be submitted on the Exercise 2 Quiz instance on Moodle with your completed notebook (showing **both your solutions and the results of their executions**). Only answers submitted through the Quiz are marked though. Marks can be lost if the notebook does not **show evidence** for the reported answers in the quiz.

While students are asked to submit their solutions through a Quiz, marking will be done with a “human-in-the-loop” and partial marks are awarded depending on the quality of the submitted work.

Your answers to the Quiz questions along with your .ipynb notebook file (showing code and outputs) must be submitted by the stated Exercise 2 deadline.

In [ ]:
import matplotlib.pyplot as plt
feature_names = ["PL2", "BM25-title", "PageRank", "URL Type", "Proximity"]

# Get the booster object
booster = lmart_5f.booster_

# Plot with correct labels
lgb.plot_importance(booster, importance_type='gain', max_num_features=5)
plt.title("Feature Importance (Gain)")
plt.xticks(fontsize=12)
plt.yticks(range(len(feature_names)), feature_names, fontsize=12)  # Assign correct labels
plt.show()
print(train_features_with_proximity[0])